In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [33]:
import tensorflow as tf
tf.__version__

'1.14.0'

# Part - 1: Prepare the data

In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
# add image filepath to the dataframe
all_xray_df = pd.read_csv('../input/data/Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in glob(os.path.join('..', 'input', 'data', 'images*', '*', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df.sample(3)

FileNotFoundError: [Errno 2] File b'../input/data/Data_Entry_2017.csv' does not exist: b'../input/data/Data_Entry_2017.csv'

In [ ]:
# visualizing label counts
label_counts = all_xray_df['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
# change the labels to one hot encoded values
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
from itertools import chain
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
# all_xray_df['disease_vec'] = all_xray_df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])
all_xray_df.sample(3)

In [ ]:
# use only some of the columns
save_columns = ['Image Index', 'Finding Labels', 'path', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
                'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
                'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
updated_xray_df = all_xray_df.loc[:, save_columns]
updated_xray_df.sample(3)

In [ ]:
# # generate train_val dataframe and save it to csv
# train_val_image_indexes = []
# with open('../input/data/train_val_list.txt') as file:
#     for line in file:
#         line = line.strip()
#         train_val_image_indexes.append(line)
        
# train_val_list_df = updated_xray_df[updated_xray_df['Image Index'].map(lambda x: True if x in train_val_image_indexes else False)]
# # train_val_list_df = train_val_list_df.drop(columns=['Finding Labels', 'path'])
# train_val_list_df.to_csv(os.path.join('../working', 'train_val_list.csv'), index=False)

In [ ]:
# # generate train_val dataframe and save it to csv
# test_image_indexes = []
# with open('../input/data/test_list.txt') as file:
#     for line in file:
#         line = line.strip()
#         test_image_indexes.append(line)
        
# test_list_df = updated_xray_df[updated_xray_df['Image Index'].map(lambda x: True if x in test_image_indexes else False)]
# # test_list_df = test_list_df.drop(columns=['Finding Labels', 'path'])
# test_list_df.to_csv(os.path.join('../working', 'test_list.csv'), index=False)

In [ ]:
# print('train_val:', train_val_list_df.shape[0], '\ttest:', test_list_df.shape[0])

In [ ]:
# move training image to image/train_val directory
# import shutil
# dst_path = '../input/data/images/train_val'
# for image_index in train_val_list_df['Image Index']:
#     try:
#         shutil.move(image_index, dest_path)
#     except:
#         print("image file conflict occured.")

In [ ]:
# split the dataset into 92% training, 5% validation and 3% test
image_indexes = np.array(updated_xray_df['Image Index'])

train_len = int(len(image_indexes) * 0.92)
val_len = int(len(image_indexes) * 0.05)
test_len = int(len(image_indexes) * 0.03)

np.random.shuffle(image_indexes)
train_indexes = image_indexes[:train_len]
val_indexes = image_indexes[train_len: train_len + val_len]
test_indexes = image_indexes[train_len + val_len: ]

train_path = "../working/train.csv"
val_path = "../working/val.csv"
test_path = "../working/test.csv"

if not os.path.exists(train_path):
    train_df = updated_xray_df[updated_xray_df['Image Index'].map(lambda x: True if x in train_indexes else False)]
    train_df.to_csv(train_path, index=False)
if not os.path.exists(val_path):
    val_df = updated_xray_df[updated_xray_df['Image Index'].map(lambda x: True if x in val_indexes else False)]
    val_df.to_csv(val_path, index=False)
if not os.path.exists(test_path):
    test_df = updated_xray_df[updated_xray_df['Image Index'].map(lambda x: True if x in test_indexes else False)]
    test_df.to_csv(test_path, index=False)

# Part - 2: Build the model

In [35]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.optimizers import Adam

In [36]:
# fine-tune densenet model
def create_model(input_shape=(224, 224, 3), num_classes=14, show_summary=False):
    img_input = Input(input_shape)
    densenet_model = DenseNet121(include_top=False,
                                 weights="imagenet",
                                 input_tensor=img_input,
                                 input_shape=input_shape,
                                 pooling="avg"
                                )
    x = densenet_model.output
    predictions = Dense(num_classes, activation="softmax", name="predictions")(x)
    model = Model(inputs=img_input, outputs=predictions)
    if show_summary:
        model.summary()

    return model

# Part - 3: Train the model

In [37]:
# check output_dir, create it if not exists
output_dir = "output/"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)
    os.makedirs(os.path.join(output_dir, "models"))
    os.makedirs(os.path.join(output_dir, "weights"))
    os.makedirs(os.path.join(output_dir, "logs"))

In [38]:
# training configurations
train_path = "../working/train.csv"
val_path = "../working/val.csv"
class_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis',
               'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

target_size=(224,224)
input_size=(224,224,3)
batch_size=32
epochs=100
initial_learning_rate = 0.001

# train_df = pd.read_csv(train_path)
# val_df = pd.read_csv(val_path)

# train_steps = np.ceil(len(train_df)/batch_size)
# validation_steps = np.ceil(len(val_df)/batch_size)

output_dir = "output/"
model_path = os.path.join(output_dir, "models/chest_xray_imaging_model.h5")
weights_path = os.path.join(output_dir, "weights/chest_xray_imaging_weights.h5")
log_dir = os.path.join(output_dir, "logs")

In [39]:
from PIL import Image
from tensorflow.keras.utils import Sequence

# define custom datagenerator
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, dataset_csv_file, class_names, batch_size=32, target_size=(224,224,3), shuffle=True):
        'Initialization'
        self.dataset_df = pd.read_csv(dataset_csv_file)
        self.indexes = np.array(self.dataset_df['Image Index'])
        self.image_paths = self.dataset_df['path']
        self.labels = self.dataset_df[class_names]
        self.batch_size = batch_size
        self.steps = np.ceil(len(self.dataset_df)/self.batch_size)
        self.target_size = target_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_x = np.asarray([self.load_image(image_path) for image_path in batch_image_paths])
        batch_x = self.normalize_batch_images(batch_x)
        batch_y = self.labels[index * self.batch_size: (index + 1) * self.batch_size]
        return batch_x, batch_y


    def load_image(self, image_path):
        image_path = os.path.join(os.getcwd(), image_path)
        image = Image.open(image_path)
        image_array = np.asarray(image.convert("RGB"))
        image_array = image_array / 255.
        image_array = np.resize(image_array, self.target_size)
        return image_array

    def normalize_batch_images(self, batch_x):
        imagenet_mean = np.array([0.485, 0.456, 0.406])
        imagenet_std = np.array([0.229, 0.224, 0.225])
        batch_x = (batch_x - imagenet_mean) / imagenet_std
        return batch_x
    
    def get_y_true(self):
        if self.shuffle:
            raise ValueError("shuffle value cannot be true when accessing y_true") 
        return self.labels[:self.steps*self.batch_size, :]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [10]:
# create data generator
train_datagen = DataGenerator(dataset_csv_file = train_path,
                              class_names = class_names,
                              batch_size=batch_size,
                              target_size=target_size,
                              shuffle=True)

validation_datagen = DataGenerator(dataset_csv_file = val_path,
                                  class_names = class_names,
                                  batch_size=batch_size,
                                  target_size=target_size,
                                  shuffle=True)

FileNotFoundError: [Errno 2] File b'../working/train.csv' does not exist: b'../working/train.csv'

In [9]:
# callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1,
                              verbose=1, mode="min", min_lr=1e-8)

tensorboard = TensorBoard(log_dir=log_dir)

callbacks = [checkpoint, reduceLR, tensorboard]

In [9]:
# train the model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import multi_gpu_model

# resum training if exists
if os.path.exists(model_path):
    # load model checkpoint
    print(".... loading trained model ....")
    model = load_model(model_path) # load the architecture, weigths, triaining configuration, and training status
else:
    print("create new model")
    model = create_model()
    model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy",  metrics=["accuracy"])

# check multiple gpu availability
gpus = len(os.getenv("CUDA_VISIBLE_DEVICES", "1").split(","))
if gpus > 1:
    print(f"** multi_gpu_model is used! gpus={gpus} **")
    model = multi_gpu_model(model, gpus)
else:
    print("there is no gpu in this device")
model.fit_generator(train_datagen, 
                    steps_per_epoch=train_datagen.steps,
                    epochs=epochs, 
                    verbose=1, 
                    callbacks=callbacks, 
                    validation_data=validation_datagen, 
                    validation_steps=validation_steps, 
                    validation_freq=1, 
                    class_weight=None, 
                    max_queue_size=10, 
                    workers=1, 
                    use_multiprocessing=False, 
                    shuffle=True, 
                    initial_epoch=0)


# Part - 4: Analyze the model’s results 

In [40]:
# testing configurations
test_path = "../working/test.csv"
target_size = (224, 224)
input_shape = (224, 224, 3)

In [ ]:
# auroc test
model = get_model(model_path)

test_datagen = DataGenerator(dataset_csv_file = test_path,
                             class_names = class_names,
                             batch_size=batch_size,
                             target_size=target_size,
                             shuffle=False)

print("** make prediction **")
y_pred = model.predict_generator(test_datagen, verbose=1)
# y = pd.read_csv(test_path).loc[:, class_names] # are they the same shape
y = test_datagen.get_y_true()

test_log_path = os.path.join(output_dir, "test.log")
print(f"** write log to {test_log_path} **")
aurocs = []
with open(test_log_path, "w") as f:
    for i in range(len(class_names)):
        try:
            score = roc_auc_score(y[:, i], y_pred[:, i])
            aurocs.append(score)
        except ValueError:
            score = 0
        f.write(f"{class_names[i]}: {score}\n")
    mean_auroc = np.mean(aurocs)
    f.write("-------------------------\n")
    f.write(f"mean auroc: {mean_auroc}\n")
    print(f"mean auroc: {mean_auroc}")

In [42]:
from tensorflow.keras.models import load_model
def get_model(model_path):
    global model
    model = load_model(model_path)
    print(" * model loaded!")

In [43]:
# load the image
from PIL import Image
from skimage.transform import resize

def load_image(image, target_size = (224, 224)): 
    # image = PIL image
    image_array = np.asarray(image.convert("RGB"))
    image_array = image_array / 255.
    image_array = resize(image_array, target_size)
    return image_array

In [44]:
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

def preprocess_image(image, target_size=(224, 224)):
    # image = PIL image
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    return image

In [45]:
def get_output_layer(model, layer_name):
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

In [70]:
import cv2
from tensorflow.keras import backend as kb
def heatmap(model, img_ori, img_transformed, predicted_class_index):
    # build tensorflow computational graph
    class_weights = model.layers[-1].get_weights()[0] # class_weight.shape = (1024, 14)
    final_conv_layer = get_output_layer(model, "bn")
    get_output = kb.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
    [conv_outputs, predictions] = get_output([np.array([img_transformed])])
    conv_outputs = conv_outputs[0, :, :, :] # conv_outputs:  (7, 7, 1024) and predictions: (1, 14)

    # Create the class activation map.
    cam = np.zeros(dtype=np.float32, shape=(conv_outputs.shape[:2]))
    for i, w in enumerate(class_weights[predicted_class_index]):
        cam += w * conv_outputs[:, :, i]
       
    # merge the original image and the class activation map
    cam /= np.max(cam)
    cam = cv2.resize(cam, img_ori.shape[:2])
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap[np.where(cam < 0.2)] = 0
    heatmap = heatmap * 0.5 + img_ori
    heatmap = (heatmap/np.max(heatmap))*255 # normalizing

    cv2.imwrite("output/heatmap.jpg", heatmap)
    return heatmap, class_weights

In [75]:
import cv2
from PIL import Image
def predict(model, image, target_size=(224, 224), show_heatmap=False):
    processed_image = preprocess_image(image, target_size)
    prediction = model.predict(processed_image).tolist()
    if show_heatmap:
        img_ori = np.asarray(image.convert("RGB"))
        img_transformed = load_image(image)
        predicted_class_index = np.argmax(prediction)
        heatmap_img = heatmap(model, img_ori, img_transformed, predicted_class_index)
        return (prediction, heatmap_img)
    else:
        return prediction

In [47]:
model_path = os.path.join(output_dir, "models/chest_xray_imaging_model.h5")
print(" * Trained model loading.... ")
get_model(model_path)
model.summary()

 * Trained model loading.... 
 * model loaded!
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_____________________________________________

In [76]:
input_dir = "input/"
image_path = os.path.join(input_dir, "test_images/00028173_003.png")

image = Image.open(image_path)
(prediction, heatmap) = predict(model, image, (224, 224), True)

In [82]:
predicted_class_index = np.argmax(prediction)
predicted_class_name = class_names[predicted_class_index]
predicted_class_value = np.max(prediction)
print("The model predict {} with {} confidence".format(predicted_class_name, predicted_class_value))

The model predict Pleural_Thickening with 0.998526930809021 confidence


In [98]:
image = Image.open(image_path)
image = image.convert("RGB")
np.array(image).shape

(1024, 1024, 3)

In [76]:
# heatmap??
heatmap = heatmap(model, image)

TypeError: 'numpy.ndarray' object is not callable

In [75]:
type(image)

PIL.PngImagePlugin.PngImageFile

In [96]:
y_pred = predict(model, image, class_names)
np.array(y_pred).shape

(1, 14)

In [91]:
y = np.zeros(shape=np.array(y_pred).shape)
y.shape

(1, 14)

In [98]:
y[0, np.argmax(y_pred)] = 1
y

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [ ]:
score = roc_auc_score(y[:, i], y_pred[:, i])

In [102]:
# aurocs test
from sklearn.metrics import roc_auc_score
for i in range(len(class_names)):
    try:
        score = roc_auc_score(y[:, i], y_pred[:, i])
        aurocs.append(score)
    except ValueError:
        score = 0
mean_auroc = np.mean(aurocs)
print(f"mean auroc: {mean_auroc}")

ModuleNotFoundError: No module named 'sklearn'

In [103]:
import sklearn

ModuleNotFoundError: No module named 'sklearn'